# Welcome to the LyubomirT's Toxicity Detector model notebook!

This notebook trains and runs a lightweight BERT-based model for detecting toxicity in text. The model is trained on the Severity of Toxic Comments dataset, which is a binary classification task. The model is trained using the `transformers` library and `torch`.

To train the model yourself, please run all the cells from the beginning. Note that you will need a GPU to train the model in a reasonable amount of time.

## Quick inference

If you just want to see the model in action, you can skip the training and jump straight to the "Inference" section. There, you can input your own text and see the model's predictions. Run all cells except the "Training" section.

Note that you need to download the trained model weights from the release page and upload them to the notebook. The model weights are available at [this link](https://github.com/LyubomirT/bert-toxicity-detection-model/releases/download/v1.0.0/model_weights.pth).

In [ ]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install tqdm
!pip install scikit-learn

# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.nn.functional as F
from tqdm import tqdm
import os

# Read and preprocess the data

First, we need to load the data and preprocess it. We will use the `pandas` library to load the data and preprocess it. In the dataset, we have labels for toxicity such as `toxic`, `severe_toxic`, `obscene`, `threat`, and `insult`. We will use all of these labels to train the model.

In [ ]:
# Cell 2: Data Loading and Preprocessing
# Load the dataset
df = pd.read_csv('train.csv')

# Define the labels we want to predict
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult']

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['comment_text'].tolist(),
    df[labels].values,
    test_size=0.2,
    random_state=42
)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# The dataset

The dataset we will use is the "Severity of Toxic Comments" dataset. It is a binary classification task where the goal is to predict whether a comment is toxic or not. The dataset contains comments from Wikipedia's talk page edits. Each comment is labeled with one or more of the following labels: `toxic`, `severe_toxic`, `obscene`, `threat`, `insult`, and `identity_hate`.

In [ ]:
# Cell 3: Dataset and DataLoader
class ToxicDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.FloatTensor(label)
        }

# Create Dataset objects
train_dataset = ToxicDataset(train_texts, train_labels, tokenizer)
val_dataset = ToxicDataset(val_texts, val_labels, tokenizer)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Definition

This notebook uses the `transformers` library to load a pre-trained BERT model and fine-tune it on the Severity of toxic comments dataset. The model is defined as a class `ToxicClassifier` that inherits from `torch.nn.Module`. The model uses the `BertForSequenceClassification` class from the `transformers` library to load a pre-trained BERT model with a classification head. The model is initialized with the number of classes in the dataset.

In [ ]:
# Cell 4: Model Definition
class ToxicClassifier(torch.nn.Module):
    def __init__(self, n_classes):
        super(ToxicClassifier, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=n_classes)
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return output.logits

# Initialize the model
model = ToxicClassifier(len(labels))

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training Loop

This is the training loop of the LyubomirT-Toxic-Detector model, here we train the model using the AdamW optimizer with a learning rate of 2e-5, a step size of 1, and a gamma of 0.1. We use the binary cross-entropy loss function and the mixed precision training technique to speed up the training process. We also use gradient accumulation to reduce the memory usage during training.

In [ ]:
# Cell 5: Training Loop
def train_model(model, train_loader, val_loader, epochs=3, accumulation_steps=2):
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    scaler = torch.cuda.amp.GradScaler()  # Mixed Precision Training
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)  # Learning Rate Scheduler

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        optimizer.zero_grad()
        for i, batch in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}')):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.cuda.amp.autocast():  # Mixed Precision Training
                outputs = model(input_ids, attention_mask)
                loss = F.binary_cross_entropy_with_logits(outputs, labels)
            
            scaler.scale(loss).backward()
            
            # Gradient accumulation
            if (i + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        predictions = []
        true_labels = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                with torch.cuda.amp.autocast():  # Mixed Precision Training
                    outputs = model(input_ids, attention_mask)
                    loss = F.binary_cross_entropy_with_logits(outputs, labels)
                
                val_loss += loss.item()

                predictions.extend(torch.sigmoid(outputs).cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch + 1}/{epochs}:')
        print(f'Train Loss: {avg_train_loss:.4f}')
        print(f'Validation Loss: {avg_val_loss:.4f}')

        scheduler.step()  # Update the learning rate

# Adjust the DataLoader for better performance
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# Train the model
train_model(model, train_loader, val_loader)


# Inference

To run inference without training, just run all cells except the one with the training loop. The model is already trained and saved as `lyubomirt-toxicity-detector.pth`. Feel free to input your own text in the `text` variable to see the model's output.

Also note that the model doesn't require a GPU to run inference, so you can run the code on a CPU. It's still pretty fast, my 14-core CPU takes around 0.1 seconds to process a single text.

In [ ]:
# Define the path to save/load the model
MODEL_PATH = 'lyubomirt-toxicity-detector.pth'
model = ToxicClassifier(len(labels))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Cell 6: Model Initialization
def initialize_model():
    # Check if the model file exists
    if os.path.exists(MODEL_PATH):
        # Load the existing model
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
        model.eval()
        print("Model loaded from file.")
    else:
        # Initialize and train your model here (this is just a placeholder)
        # For example: model = YourModelClass()
        # Train the model
        # Save the model after training
        torch.save(model.state_dict(), MODEL_PATH)
        print("Model saved to file.")

# Call this function once to initialize the model
initialize_model()

# Cell 6: Inference Function
def predict_toxicity(text):
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
    
    probabilities = torch.sigmoid(outputs).cpu().numpy()[0]
    predictions = (probabilities > 0.5).astype(int)
    
    result = {}
    for label, prob, pred in zip(labels, probabilities, predictions):
        result[label] = {'probability': float(prob), 'prediction': int(pred)}
    
    return result

# Example usage
text = input("Enter text: ")
result = predict_toxicity(text)
print(f"Input text: {text}")
for label, values in result.items():
    print(f"{label}: Probability = {values['probability']:.4f}, Prediction = {values['prediction']}")